In [179]:
import numpy as np

# Load the data

For $k = 0, 1, 2$ we have the following files:
* Xtrk.csv - the training sequences.
* Xtek.csv - the test sequences.
* Ytrk.csv - labels for the training sequences

In [180]:
Xtr0_mat100 = np.genfromtxt("data/Xtr0_mat100.csv", delimiter='')
Ytr0 = np.genfromtxt("data/Ytr0.csv", delimiter=',', skip_header=1)

Xtr1_mat100 = np.genfromtxt("data/Xtr1_mat100.csv", delimiter='')
Ytr1 = np.genfromtxt("data/Ytr1.csv", delimiter=',', skip_header=1)

Xtr2_mat100 = np.genfromtxt("data/Xtr2_mat100.csv", delimiter='')
Ytr2 = np.genfromtxt("data/Ytr2.csv", delimiter=',', skip_header=1)


In [193]:
Ytr2[:,1].min()

0.0

In [213]:
def accuracy(y_true,y_pred):
    n = y_true.shape[0]
    predictions = np.zeros(n)
    predictions[y_pred >= 0.5] = 1
    return np.sum(y_true == predictions) / n

# Implementing some kernels

## Linear Kernel

In [182]:
def linear_kernel(X):
    return(X @ X.T)

## Gaussian Kernel

In [183]:
def gaussian(x,y, sigma):
    exp_term = np.linalg.norm(x-y)**2 /(2*sigma)
    return(np.exp(-exp_term))

# Naive computation of the gaussian kernel that can be easily improved
def gaussian_kernel(X,sigma):
    n = X.shape[0]
    K = np.eye(n) # One along the diagonals because K(x,x) = exp(0) = 1
    for i in range(n):
        for j in range(i+1,n):
            val = gaussian(X[i], X[j], sigma)
            K[i,j] = val
            K[j,i] = val
    return(K)
    

## Polynomial Kernel

## Kernel Ridge Regression

* Consider RKHS $\mathcal H$, associated to a p.d. kernel K on $\mathcal X$
* Let $y = (y_1, \dots, y_n)^T \in \mathbb R ^n$
* Let $\alpha = (\alpha_1, \dots, \alpha_n)^T \in \mathbb R ^n$
* Let $K$ be the $n\times n$ Gram Matrix such that $K_{i,j} = K(x_i, x_j)$
* We can then write
$$
(\hat f(x_1), \dots, \hat f(x_n))^T = K\alpha
$$
* The norm is $||\hat f||^2_{\mathcal H} = \alpha^T K \alpha$
* KRR $\leftrightarrow \text{argmin}_{\alpha \in \mathbb R^n} \frac{1}{n} (K\alpha - y)^T(K\alpha - y) + \lambda \alpha^T K \alpha$
* Solution for $\lambda > 0$:
$$
\alpha = (K+\lambda nI)^{-1}y
$$


In [255]:
def KRR(K, y, lambd):
    """
    takes the kernel matrix as an input and computes the MSE and the predictions for each value in lambd (list)
    """
    assert K.shape[0] == y.shape[0]
    assert len(lambd) > 0
    
    y_preds = []
    loss = []
    accuracies = []
    alphas = []
    for l in lambd:
        assert l >= 0
        # find the parameter alpha
        alpha = np.linalg.solve((K + l*n*np.eye(n)), y)
        # predict
        loss_lambda = MSE(K, y, l, alpha)
        accuracy_lambda = accuracy(y,K@alpha)
        print(f"The MSE  and accuracy for lambda = {l} are : {loss_lambda:.4f}, {accuracy_lambda:.6f} ")
        y_preds += [K @ alpha]
        loss += [loss_lambda]
        accuracies += [accuracy_lambda]
        alphas +=[alpha]
        
    return(y_preds, alphas, loss, accuracies)
    

In [228]:
def MSE(K, y, lambd, alpha):
    n = y.shape[0]
    data_term = (np.linalg.norm(np.dot(K, alpha.reshape(-1,1)) - y)**2)/n
    reg_term = alpha @ K @ alpha
    return(data_term + lambd * reg_term)

In [229]:
K_tr0 = gaussian_kernel(Xtr0_mat100,1)

In [256]:
lambdas = [0] + [10**i for i in range(-10,2)]
pred_tr0, alphas_tr0, loss_tr0, accuracies = KRR(K_tr0, Ytr0[:,1], lambdas)

The MSE  and accuracy for lambda = 0 are : 998.5560, 1.000000 
The MSE  and accuracy for lambda = 1e-10 are : 991.7629, 1.000000 
The MSE  and accuracy for lambda = 1e-09 are : 943.8538, 1.000000 
The MSE  and accuracy for lambda = 1e-08 are : 775.9972, 0.995500 
The MSE  and accuracy for lambda = 1e-07 are : 614.2858, 0.866000 
The MSE  and accuracy for lambda = 1e-06 are : 567.3305, 0.706000 
The MSE  and accuracy for lambda = 1e-05 are : 554.5332, 0.656000 
The MSE  and accuracy for lambda = 0.0001 are : 523.9276, 0.637000 
The MSE  and accuracy for lambda = 0.001 are : 501.4996, 0.607500 
The MSE  and accuracy for lambda = 0.01 are : 499.3773, 0.519000 
The MSE  and accuracy for lambda = 0.1 are : 503.2188, 0.519000 
The MSE  and accuracy for lambda = 1 are : 616.6589, 0.519000 
The MSE  and accuracy for lambda = 10 are : 882.6979, 0.519000 


## Kernel Logistic Regression

- Binary Classificaiton setup: $\mathcal Y = \{-1, 1\}$
- $\mathcal l_{\text{logistic}}(f(x),y) = -\log p(y|f(x)) = \log(1 + e^{-yf(x)})$ where $p(y|f(x)) = \sigma(y(f(x))$

Objective:
\begin{align*}
\hat f &= \text{argmin}_{f\in \mathcal H} \frac{1}{n} \sum_{i=1}^n \log(1+e^{-y_if(x_i)}) + \frac{\lambda}{2}||f||^2_{\mathcal H}\\
\alpha &= \text{argmin}_{\alpha \in \mathbb R^n} \frac{1}{n} \sum_{i=1}^n \log(1+e^{-y_i[K\alpha]_i}) + \frac{\lambda}{2} \alpha^T K \alpha
\end{align*}

We define the following fonctions and vectors:
* $\mathcal l _\text{logistic}(u) = \log(1+e^{-u})$
* $\mathcal l' _\text{logistic}(u) = -\sigma(-u)$
* $\mathcal l'' _\text{logistic}(u) = \sigma(u)\sigma(-u)$

* for $i = 1, \dots, n$, $P_i(\alpha) = \mathcal l' _\text{logistic}(y_i[K\alpha]_i)$
* for $i = 1, \dots, n$, $W_i(\alpha) = \mathcal l'' _\text{logistic}(y_i[K\alpha]_i)$




\begin{align*}
J(\alpha) &= \frac{1}{n} \sum_{i=1}^n \log(1+e^{-y_i[K\alpha]_i}) + \frac{\lambda}{2} \alpha^T K \alpha\\
\nabla J(\alpha) &= \frac{1}{n} KP(\alpha) y + \lambda K \alpha \quad \text{where } P(\alpha) = \text{diag}(P_1(\alpha), \dots, P_n(\alpha))\\
\nabla^2 J(\alpha) &= \frac{1}{n}KW(\alpha)K+\lambda K \quad \text{where } W(\alpha) = \text{diag}(W_1(\alpha), \dots, W_n(\alpha))
\end{align*}

We are interested in the quadratic approximation of $J$ near a point $\alpha_0$:
\begin{align*}
J_q(\alpha) &= J(\alpha_0) + (\alpha - \alpha_0)^T \nabla J(\alpha_0) + \frac{1}{2} (\alpha - \alpha_0)^T \nabla^2 J(\alpha_0)(\alpha - \alpha_0)\\
2J_q(\alpha) &= -\frac{2}{n} \alpha^T KW(K\alpha_0-W^{-1}Py)+\frac{1}{n}\alpha^TKWK\alpha+ \lambda\alpha^TK\alpha +C\\
&= \frac{1}{n} (K\alpha - z)^TW(K\alpha - z) + \lambda\alpha^TK\alpha + C \quad \text{where} z = K\alpha_0 - W^{-1} P y
\end{align*}

The WKRR problem is presented as:
$$
\text{argmin}_{\alpha \in \mathbb R^n} \frac{1}{n}(K\alpha - y)^TW(K\alpha - y) + \lambda \alpha^TK\alpha
$$
and has as :
$$
\alpha = W^{1/2} (W^{1/2}KW^{1/2}+n\lambda I)^{-1} W^{1/2}y
$$

So, in order to solve KRL, we use IRLS on a WKRR problem until convergence:
$$\alpha^{t+1} \gets \text{solveWKRR}(K, W^t, z^t)$$
With the updates for $W^t$ and $z^t$ from $\alpha^t$ are:
- $m_i \gets [K\alpha^t]_i$
- $P_i^t \gets -\sigma(-y_im_i)$
- $W_i^t \gets \sigma(m_i)\sigma(-m_i)$
- $z_i^t \gets m_i + y_i / \sigma(-y_im_i)$

In [231]:
def solveWKRR(K,W,z,y,lambd):
    
    assert np.all(W >= 0)
    
    W_sq = np.sqrt(W)
    n = K.shape[0]
    inv_matrix = np.linalg.solve((W_sq @ K @ W_sq + n * lambd * np.eye(n)), W_sq @ y)
    alpha = W_sq @ inv_matrix
    
    return alpha
    

In [232]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def logistic_loss(y_true, y_pred):
    n = y_true.shape[0]
    log_term = np.log(sigmoid(y_true*y_pred))
    return(-np.sum(log_term)/n)
    

In [265]:
def KLR(K, y, lambd, maxIter = 100, tresh = 1e-8):
    
    # initialize the values
    assert K.shape[0] == y.shape[0]
    n = K.shape[0]
    
    y_preds = []
    loss = []
    accuracies = []
    alphas = []
    
    for l in lambd :
        cnt = 0
        
        P_t, W_t = np.eye(n), np.eye(n)
        z_t = K@ np.ones(n) - y
        alpha_t = np.ones(n)
        diff_alpha = np.inf


        while (diff_alpha > tresh) and (cnt < maxIter):

            old_alpha = alpha_t
            alpha_t = solveWKRR(K, W_t, z_t, y, l)

            m_t = K@alpha_t
            sigma_m = sigmoid(m_t)
            sigma_my = sigmoid(-y*m_t)

            P_t = - np.diag(sigma_my)
            W_t = np.diag(sigma_m * (1-sigma_m))

            z_t = m_t - (P_t@y)/(sigma_m * (1-sigma_m))

            diff_alpha = np.linalg.norm(alpha_t - old_alpha)
            cnt+=1
            if cnt % 10 == 0:
                print(l, cnt)

        pred_l = K@alpha_t
        y_preds += [pred_l]
        loss_l = logistic_loss(y, pred_l)
        loss += [loss_l]
        accuracy_l = accuracy(y, pred_l)
        accuracies += [accuracy_l]
        alphas +=[alpha_t]
        print(f"The logistic loss and accuracy for lambda = {l} are : {loss_l:.4f}, {accuracy_l:.6f} ")
        
    
    return y_preds, alphas, loss, accuracies
        

In [251]:
lambdas = [0] + [10**i for i in range(-10,2)]
preds = KLR(K_tr0, Ytr0[:,1], lambdas)

0 10
0 20
0 30
0 40
0 50
0 60
0 70
0 80
0 90
0 100
The logistic loss and accuracy for lambda = 0 are : 0.5104, 1.000000 
1e-10 10
1e-10 20
1e-10 30
1e-10 40
1e-10 50
1e-10 60
1e-10 70
1e-10 80
1e-10 90
1e-10 100
The logistic loss and accuracy for lambda = 1e-10 are : 0.5126, 1.000000 
1e-09 10
1e-09 20
1e-09 30
1e-09 40
1e-09 50
1e-09 60
1e-09 70
1e-09 80
1e-09 90
1e-09 100
The logistic loss and accuracy for lambda = 1e-09 are : 0.5245, 0.998500 
1e-08 10
1e-08 20
1e-08 30
1e-08 40
1e-08 50
1e-08 60
1e-08 70
1e-08 80
1e-08 90
1e-08 100
The logistic loss and accuracy for lambda = 1e-08 are : 0.5522, 0.941000 
1e-07 10
1e-07 20
1e-07 30
1e-07 40
1e-07 50
1e-07 60
1e-07 70
1e-07 80
1e-07 90
1e-07 100
The logistic loss and accuracy for lambda = 1e-07 are : 0.5730, 0.746500 
1e-06 10
The logistic loss and accuracy for lambda = 1e-06 are : 0.5797, 0.665500 
The logistic loss and accuracy for lambda = 1e-05 are : 0.5829, 0.648500 
The logistic loss and accuracy for lambda = 0.0001 are : 0.588

# Testing the accuracy

## Create the kernel matrices

In [252]:
K_tr0 = gaussian_kernel(Xtr0_mat100,1)
K_tr1 = gaussian_kernel(Xtr1_mat100,1)
K_tr2 = gaussian_kernel(Xtr2_mat100,1)

## Testing KRR

In [262]:
lambdas = [0] + [10**i for i in range(-10,2)]
print("*************KRR for dataset 0*************\n")
pred_tr0, alphas_tr0, loss_tr0, accuracies_0 = KRR(K_tr0, Ytr0[:,1], lambdas)
print("*************KRR for dataset 1*************\n")
pred_tr1, alphas_tr1, loss_tr1, accuracies_1 = KRR(K_tr1, Ytr1[:,1], lambdas)
print("*************KRR for dataset 2*************\n")
pred_tr2, alphas_tr2, loss_tr2, accuracies_2 = KRR(K_tr2, Ytr2[:,1], lambdas)

*************KRR for dataset 0*************

The MSE  and accuracy for lambda = 0 are : 998.5560, 1.000000 
The MSE  and accuracy for lambda = 1e-10 are : 991.7629, 1.000000 
The MSE  and accuracy for lambda = 1e-09 are : 943.8538, 1.000000 
The MSE  and accuracy for lambda = 1e-08 are : 775.9972, 0.995500 
The MSE  and accuracy for lambda = 1e-07 are : 614.2858, 0.866000 
The MSE  and accuracy for lambda = 1e-06 are : 567.3305, 0.706000 
The MSE  and accuracy for lambda = 1e-05 are : 554.5332, 0.656000 
The MSE  and accuracy for lambda = 0.0001 are : 523.9276, 0.637000 
The MSE  and accuracy for lambda = 0.001 are : 501.4996, 0.607500 
The MSE  and accuracy for lambda = 0.01 are : 499.3773, 0.519000 
The MSE  and accuracy for lambda = 0.1 are : 503.2188, 0.519000 
The MSE  and accuracy for lambda = 1 are : 616.6589, 0.519000 
The MSE  and accuracy for lambda = 10 are : 882.6979, 0.519000 
*************KRR for dataset 1*************

The MSE  and accuracy for lambda = 0 are : 999.9990,

## Testing KLR

In [267]:
lambdas = [0] + [10**i for i in range(-10,0)]
print("*************KLR for dataset 0*************\n")
pred_klr_tr0, alphas_klr_tr0, loss_klr_tr0, accuracies_klr_0 = KLR(K_tr0, Ytr0[:,1], lambdas, tresh=1e-5)
print("*************KLR for dataset 1*************\n")
pred_klr_tr1, alphas_klr_tr1, loss_klr_tr1, accuracies_klr_1 = KLR(K_tr1, Ytr1[:,1], lambdas, tresh=1e-5)
print("*************KLR for dataset 2*************\n")
pred_klr_tr2, alphas_klr_tr2, loss_klr_tr2, accuracies_klr_2 = KLR(K_tr2, Ytr2[:,1], lambdas, tresh=1e-5)

*************KLR for dataset 0*************

0 10
0 20
0 30
0 40
0 50
0 60
0 70
0 80
0 90
0 100
The logistic loss and accuracy for lambda = 0 are : 0.5104, 1.000000 
1e-10 10
1e-10 20
1e-10 30
1e-10 40
1e-10 50
1e-10 60
1e-10 70
1e-10 80
1e-10 90
1e-10 100
The logistic loss and accuracy for lambda = 1e-10 are : 0.5126, 1.000000 
1e-09 10
1e-09 20
1e-09 30
1e-09 40
1e-09 50


KeyboardInterrupt: 